In [1]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)
//Double negation theorem --> DNx : Not(Not[X]) => X
//Law of the third middle --> TMx : Either[X,Not[X]] 

defined type Not
defined type <=>

## 1. Natural deduction
$$
\begin{array}{c} 
& T [ p \rightarrow q, \neg r \rightarrow \neg q, r \rightarrow \neg s]\vdash \neg s \vee \neg p & 
\end{array}
$$

$$
\begin{array}{llr}
1 & ((p \rightarrow q) \wedge (\neg r \rightarrow \neg q)) \wedge (r \rightarrow \neg s) & premise \\
1.1 & p \rightarrow q & \wedge E(\wedge E(1)) \\
1.2 & \neg r \rightarrow \neg q & \wedge E(\wedge E(1)) \\
1.3 & r \rightarrow \neg s & \wedge E(1) \\
1.4 & p \vee \neg p & LEM \\
&& \\
1.5 & \neg p & \vee E(1.4)\\
1.5.1 & \neg s \vee \neg p & \vee I(1.5) \\
&& \\
1.6 & p & \vee E(1.4) \\
1.6.1 & r \vee \neg r & LEM \\
&&\\
1.6.2 & r & \vee E(1.6.1) \\
1.6.2.1 & \neg s & \rightarrow E(1.3,1.6.2) \\
1.6.2.2 & \neg s \vee \neg p & \vee I(1.6.2.1) \\
&&\\
1.6.3 & \neg r & \vee E(1.6.1)\\
1.6.3.1 & q & \rightarrow E(1.1,1.6) \\
1.6.3.2 & \neg q & \rightarrow E(1.2,1.6.3) \\
1.6.3.3 & ⊥ & \rightarrow I(16.3.2,1.6.3.1)\\
1.6.3.4 & \neg s \vee \neg p & ⊥E(1.6.3.3) \\
&& \\
1.6.4 & \neg s \vee \neg p & \vee E(1.6.2-1.6.2.2, 1.6.3-1.6.3.4, 1.6.1)\\
&& \\
1.7 & \neg s \vee \neg p & \vee E(1.5-1.5.1, 1.6-1.6.4, 1.4)\\
2 & ((p \rightarrow q) \wedge (\neg r \rightarrow \neg q)) \wedge (r \rightarrow \neg s) \rightarrow \neg s \vee \neg p & \rightarrow I(1-1.7) \\
\end{array}
$$

In [2]:
trait LEM{
    def apply[P]: Either[P, Not[P]]
}

object LEMobj{
    type Prop[P] = Either[P, Not[P]]
}

defined trait LEM
defined object LEMobj

In [3]:
def proof[P,Q,R,S](lem: LEM): (((P => Q, Not[R] => Not[Q]), R => Not[S])) => Either[Not[S],Not[P]] = 
    (_1: ((P => Q, Not[R] => Not[Q]), R => Not[S])) => {               // 1. ((P -> Q) ^ (¬R -> ¬Q)) ^ (R -> ¬S)  
        val _1_1: P => Q = _1._1._1                                    // 1.1. P -> Q                                             ^E(^E(1))
        val _1_2: Not[R] => Not[Q] = _1._1._2                          // 1.2. ¬R -> ¬Q                                           ^E(^E(1))
        val _1_3: R => Not[S] = _1._2                                  // 1.3. R -> ¬S                                            ^E(1)
        val _1_4: LEMobj.Prop[P] = lem[P]                              // 1.4. P v ¬P                                             LEM
        _1_4 match {                                               
            case Right(_1_5: Not[P]) =>                                // 1.5. ¬P                            
                Right(_1_5: Not[P])                                    // 1.5.1 ¬S v ¬P                                           vI(1.5)
            
            case Left(_1_6: P) =>                                      // 1.6. P 
                val _1_6_1: LEMobj.Prop[R] = lem[R]                    // 1.6.1. R v ¬R                                           LEM 
                _1_6_1 match {                                         
                    case Left(_1_6_2: R) =>                            // 1.6.2. R                                   
                        val _1_6_2_1: Not[S] = _1_3(_1_6_2)            // 1.6.2.1. ¬S                                             ->E(1.3, 1.6.2)
                        Left(_1_6_2_1: Not[S])                         // 1.6.2.2. ¬S v ¬P                                        vI(1.6.2.1)
                    
                    case Right(_1_6_3: Not[R]) =>                      // 1.6.3. ¬R
                        val _1_6_3_1: Q = _1_1(_1_6)                   // 1.6.3.1. Q                                              ->E(1.1, 1.6)
                        val _1_6_3_2: Not[Q] = _1_2(_1_6_3)            // 1.6.3.2. ¬Q                                             ->E(1.2, 1.6.3)
                        val _1_6_3_3: Nothing = _1_6_3_2(_1_6_3_1)     // 1.6.3.3. ⊥                                              ->E(1.6.3.2, 1.6.3.1)
                        _1_6_3_3                                       // 1.6.3.4. ¬S v ¬P                                        ⊥E(1.6.3.3)
                    
                }                                                      // 1.6.4. ¬S v ¬P                                          vE(1.6.2-1.6.2.2, 1.6.3-1.6.3.4, 1.6.1)
            
        }                                                              // 1.7. ¬S v ¬P                                            vE(1.5-1.5.1, 1.6-1.6.4, 1.4)
    }                                                                  // 2. ((P -> Q) ^ (¬R -> ¬Q)) ^ (R -> ¬S) -> ¬S v ¬P       ->I(1, 1.7)

defined function proof

## 1.2. Natural deduction (with auxiliary functions)
$$
\begin{array}{c} 
& T [ p \rightarrow q, \neg r \rightarrow \neg q, r \rightarrow \neg s]\vdash \neg s \vee \neg p & 
\end{array}
$$

In [4]:
trait DN:
    def left[P]: Not[Not[P]] => P
    def right[P]: P => Not[Not[P]] =
        p => np => np(p)

trait LEM:
    def apply[P]: Either[P, Not[P]]
    
def materialImplL[P, Q](f: P => Q)(using LEM: LEM): Either[Not[P], Q] =
    LEM[P].fold(
        p => Right(f(p))
        ,
        np => Left(np))

def tollensL[P, Q](f: P => Q): Not[Q] => Not[P] =
    nq => p =>
        nq(f(p))

def tollensR[P, Q](f: Not[P] => Not[Q])(using DN: DN): Q => P =
    q =>
        val nnq : Not[Not[Q]] = DN.right(q)
        val nnp : Not[Not[P]] = tollensL(f)(nnq)
        DN.left(nnp)

def antDN[P, Q](f: P => Q)(using DN: DN): Not[Not[P]] => Q =
    nnp =>
        f(DN.left(nnp))


defined trait DN
defined trait LEM
defined function materialImplL
defined function tollensL
defined function tollensR
defined function antDN

$$
\begin{array}{llr}
1 & ((p \rightarrow q) \wedge (\neg r \rightarrow \neg q)) \wedge (r \rightarrow \neg s) & premise \\
1.1 & p \rightarrow q & \wedge E(\wedge E(1)) \\
1.2 & \neg r \rightarrow \neg q & \wedge E(\wedge E(1)) \\
1.3 & r \rightarrow \neg s & \wedge E(1) \\
&& \\
1.4 & \neg \neg r \rightarrow \neg s & antDN(1.3) \\
1.5 & s \rightarrow \neg r & tollensR(1.4) \\
1.6 & \neg q \rightarrow \neg p & tollensL(1.1) \\
1.7 & s \rightarrow \neg q & andThen(1.5-1.2) \\
1.8 & s \rightarrow \neg p & andThen(1.7-1.6) \\
&& \\
1.9 & \neg s \vee \neg p & materialImplL(1.8) \\
2 & ((p \rightarrow q) \wedge (\neg r \rightarrow \neg q)) \wedge (r \rightarrow \neg s) \rightarrow \neg s \vee \neg p & \rightarrow I(1-1.9) \\
\end{array}
$$

In [5]:
def proof[P,Q,R,S](_1_1: P => Q, _1_2: Not[R] => Not[Q], _1_3: R => Not[S])(using DN: DN, LEM: LEM): Either[Not[S], Not[P]] =
    val _1_4 : Not[Not[R]] => Not[S] = antDN(_1_3)                          // 1.4. ¬¬R -> ¬S                                           antDN(1.3)
    val _1_5 : S => Not[R] = tollensR(_1_4)                                 // 1.5. S -> ¬R                                             tollensR(1.4)
    val _1_6 : Not[Q] => Not[P] = tollensL(_1_1)                            // 1.6. ¬Q -> ¬P                                            tollensL(1.5)
    val _1_7 : S => Not[Q] = _1_5 andThen _1_2                              // 1.7. S -> ¬Q                                             andThen(1.5-1.2)
    val _1_8 : S => Not[P] = _1_7 andThen _1_6                              // 1.8. S -> ¬P                                             andThen(1.7-1.6)
    val _1_9 : Either[Not[S], Not[P]] = materialImplL(_1_8)                 // 1.9. ¬S v ¬P                                             materialImplL(1.8)
    _1_9                                                                    // 2. ((P -> Q) ^ (¬R -> ¬Q)) ^ (R -> ¬S) -> ¬S v ¬P        ->I(1, 1.9)

defined function proof

## 2. Natural deduction
$$
\begin{array}{c} 
& T [ \neg p \rightarrow \neg s, \neg p \vee r, r \rightarrow \neg t]\vdash \neg s \vee \neg t & 
\end{array}
$$

$$
\begin{array}{llr}
1 & ((\neg p \rightarrow \neg s) \wedge (\neg p \vee r)) \wedge (r \rightarrow \neg t) & premise \\
1.1 & \neg p \rightarrow \neg s & \wedge E(\wedge E(1)) \\
1.2 & \neg p \vee r & \wedge E(\wedge E(1)) \\
1.3 & r \rightarrow \neg t & \wedge E(1) \\
&& \\
1.4 & \neg p & \vee E(1.2)\\
1.4.1 & \neg s & \rightarrow E(1.1,1.4) \\
1.4.2 & \neg s \vee \neg t & \vee I(1.4.1) \\
&& \\
1.5 & r & \vee E(1.2)\\
1.5.1 & \neg t & \rightarrow E(1.3,1.5) \\
1.5.2 & \neg s \vee \neg t & \vee I(1.5.1) \\
&& \\
1.6 & \neg s \vee \neg t & \vee E(1.4-1.4.2, 1.5-1.5.2, 1.2)\\
2 & ((\neg p \rightarrow \neg s) \wedge (\neg p \vee r)) \wedge (r \rightarrow \neg t) \rightarrow \neg s \vee \neg t & \rightarrow I(1-1.6) \\
\end{array}
$$

In [6]:
def proof2[P,R,S,T]: (((Not[P] => Not[S], Either[Not[P],R]), R => Not[T])) => Either[Not[S],Not[T]] =
(_1 : ((Not[P] => Not[S], Either[Not[P],R]), R => Not[T])) => {          //  1.      ((¬P -> ¬S) ^ (¬P v R)) ^ (R -> ¬T)                  PREMISE
    val _1_1 : Not[P] => Not[S] = _1._1._1                               //  1.1.    ¬P -> ¬S                                             ^E(^E(1))
    val _1_2 : Either[Not[P],R] = _1._1._2                               //  1.2.    ¬P v R                                               ^E(^E(1))
    val _1_3 : R => Not[T] = _1._2                                       //  1.3.    R -> ¬T                                              ^E(1)
    _1_2 match{
        case Left(_1_4: Not[P]) =>                                       //  1.4.    ¬P                                                   vE(1.2)
            val _1_4_1 : Not[S] = _1_1(_1_4)                             //  1.4.1.  ¬S                                                   ->E(1.1, 1.4)
            Left(_1_4_1: Not[S])                                         //  1.4.2.  ¬S v ¬T                                              vI(1.4.1)

        case Right(_1_5: R) =>                                           //  1.5.    R                                                    vE(1.2)
            val _1_5_1: Not[T] = _1_3(_1_5)                              //  1.5.1.  ¬T                                                   ->E(1.3, 1.5)
            Right(_1_5_1: Not[T])                                        //  1.5.2.  ¬S v ¬T                                              vI(1.5.1)

    }                                                                    //  1.6     ¬S v ¬T                                              vE(1.4-1.4.2, 1.5-1.5.2, 1.2)
}                                                                        //  2       ((¬P -> ¬S) ^ (¬P v R)) ^ (R -> ¬T) -> (¬S v ¬T)    ->I(1-1.6)

defined function proof2